# Multiple Linier Regression

Mari kita coba memperluas model Penjualan kita dari model regresi linear menjadi model multiple regresi. Melihat kembali scatter plot yang telah dibuat sebelumnya, kita dapat melihat bahwa Radio juga tampaknya berkorelasi linear dengan Penjualan. Jadi, mari kita coba membuat model regresi linear multipel yang terlihat seperti berikut:

Penjualan = m1 * TV + m2 * Radio + b

Untuk melakukan ini dengan https://github.com/sajari/regression, kita hanya perlu memberi label variabel lain dalam nilai regression.Regression dan memastikan nilai-nilai ini dipasangkan dalam titik data pelatihan. Kita kemudian akan menjalankan regresi dan melihat bagaimana rumusnya keluar.

In [1]:
import "github.com/sajari/regression"

In [2]:
func training(file string) (regression.Regression, error) {
    var r regression.Regression 
    
    f, err := os.Open(file) 
    if err != nil { 
        return r, err 
    }
    
    defer f.Close() 
    
    reader := csv.NewReader(f)
    reader.FieldsPerRecord = -1
    
    r.SetObserved("Sales") 
    r.SetVar(0, "TV")
    r.SetVar(1, "Radio")
    
    for {
        // Read in a row. Check if we are at the end of the file. 
        record, err := reader.Read() 
        if err == io.EOF { 
            break 
        }
    
        // skip header
        if record[0] == "TV" {
            continue
        }
        
        yVal, err := strconv.ParseFloat(record[3], 64) 
        if err != nil { 
            return r, err
        }
    
        tvVal, err := strconv.ParseFloat(record[0], 64) 
        if err != nil { 
            return r, err
        }

        radioVal, err := strconv.ParseFloat(record[1], 64) 
        if err != nil { 
            return r, err
        }
        
        r.Train(regression.DataPoint(yVal, []float64{tvVal, radioVal}))
    }
    
    r.Run()

    return r, nil
}

In [3]:
%%
r, err := training("./01-shuffle-training.csv")
if err != nil {
    log.Fatal(err)
}
fmt.Printf("\nRegression Formula:\n%v\n\n", r.Formula)


Regression Formula:
Predicted = 3.0328 + TV*0.0448 + Radio*0.1866



Selanjutnya kita bisa membuat fungsi predic berdasarkan formula tersebut.

In [4]:
func predict(tv float64, radio float64) float64 {
    return 3.0328 + tv*0.0448 + radio*0.1866
}

## Evaluasi

In [5]:
func evaluasi(file string) (float64, error) {
    var mAE float64
    f, err := os.Open(file) 
    if err != nil { 
        return mAE, err
    }
    
    defer f.Close() 
    
    reader := csv.NewReader(f)
    reader.FieldsPerRecord = -1
    testData, err := reader.ReadAll() 
    if err != nil { 
        return mAE, err
    }
    
    // delete header
    testData = testData[1:]
    
    for _, record := range testData {
        yObserved, err := strconv.ParseFloat(record[3], 64) 
        if err != nil { 
            return mAE, err
        }
    
        tvVal, err := strconv.ParseFloat(record[0], 64) 
        if err != nil { 
            return mAE, err
        }

        radioVal, err := strconv.ParseFloat(record[1], 64) 
        if err != nil { 
            return mAE, err
        }
        
        // Predict y with our trained model. 
        yPredicted := predict(tvVal, radioVal)
    
        // Add the to the mean absolute error. 
        mAE += math.Abs(yObserved-yPredicted) / float64(len(testData))
    }

    return mAE, nil
}

In [6]:
%%
mAE, err := evaluasi("./01-shuffle-test.csv")
if err != nil {
    log.Fatal(err)
}

fmt.Printf("MAE = %0.2f\n\n", mAE)

MAE = 1.22



Model regresi linear multipel kita yang baru telah meningkatkan MAE (Mean Absolute Error)! Ini berarti kita sekarang berada dalam kondisi yang baik untuk memprediksi Penjualan berdasarkan pengeluaran iklan kita.

Anda juga dapat mencoba menambahkan variabel Iklan di Surat Kabar ke dalam model sebagai latihan lanjutan untuk melihat bagaimana performa model terpengaruh.

Namun penting untuk diingat:

* Semakin kompleks model yang Anda buat, semakin Anda mengorbankan kesederhanaan.
* Kompleksitas yang berlebihan dapat menyebabkan overfitting, di mana model terlalu fokus pada data latihan dan tidak dapat digeneralisasikan dengan baik ke data baru.

Oleh karena itu, Anda sebaiknya hanya menambahkan kompleksitas ke model jika peningkatan performa model memberikan nilai tambah yang nyata untuk kasus penggunaan Anda.